In [2]:
from sqlalchemy import create_engine, Column, Integer, String, inspect, DateTime, text, ForeignKey
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship
import pandas as pd



In [ ]:
class BBDD_MANAGEMENT():
    def __init__(self, database_path):
        self.engine = create_engine(f"sqlite:///{database_path}")
        with self.engine.connect() as conn:
            conn.execute("PRAGMA foreign_keys = ON;")
        self.Base = declarative_base()
        self.Session = sessionmaker(bind=self.engine)
        self.session = self.Session()
        self.tables = inspect(self.engine).get_table_names()
        self.models = self._generate_models()
            
    def _generate_models(self):
        inspector = inspect(self.engine)
        models = {}

        for table_name in self.tables:
            columns = []
            for column_info in inspector.get_columns(table_name):
                column_name = column_info['name']
                column_type = column_info['type']
                columns.append(Column(column_name, column_type, primary_key=column_info.get('primary_key', False)))

            # Crear la clase dinámica
            model = type(
                table_name.capitalize(),
                (self.Base,),
                {'__tablename__': table_name, **{col.name: col for col in columns}}
            )
            models[table_name.capitalize()] = model

        return models

    def create_table(self, table_name: str, columns: dict, relationships: dict = None, primary_key: str = 'id'):
        # verify if the table already exists
        inspector = inspect(self.engine)
        if table_name in inspector.get_table_names():
            print(f"La tabla {table_name} ya existe en la base de datos.")
            return

        # Create a class for the table
        attrs = {'__tablename__': table_name}
        for column_name, column_type in columns.items():
            if column_name == primary_key:
                attrs[column_name] = Column(column_type, primary_key=True)

            else:
                attrs[column_name] = Column(column_type)
        
        if relationships:
            for column_name, rel_info in relationships.items():
                ref_table = rel_info['reference_table']
                ref_column = rel_info['reference_column']
                attrs[column_name] = Column(columns[column_name], ForeignKey(f"{ref_table}.{ref_column}"))
                relationship_name = f"{ref_table}_relation"
                attrs[relationship_name] = relationship(ref_table.capitalize())
        
        attrs["create_date"] = Column(DateTime, nullable=False)
        attrs["update_date"] = Column(DateTime, nullable=False)

        # Create the table
        model = type(table_name.capitalize(), (self.Base,), attrs)
        self.Base.metadata.create_all(self.engine)
        self.models[table_name.capitalize()] = model

        print(f"La tabla {table_name} ha sido creada en la base de datos.")

    def drop_table(self, table_name: str):
        inspector = inspect(self.engine)
        if table_name in inspector.get_table_names():
            self.models[table_name.capitalize()].__table__.drop(self.engine)
            print(f"La tabla {table_name} ha sido eliminada de la base de datos.")
        else:
            print(f"La tabla {table_name} no existe en la base de datos.")

    def upload_data(self, table_name: str, data: dict):
        inspector = inspect(self.engine)
        

        if table_name in inspector.get_table_names():
            columns = [col['name'] for col in inspector.get_columns(table_name)]
            if "id" in columns:
                with self.engine.connect() as connection:
                    query = f"SELECT MAX(id) FROM {table_name}"
                    result = connection.execute(text(query))
                    max_id = result.scalar()
                    next_id = (max_id or 0) + 1
                data['id'] = next_id
            
            data["create_date"] = pd.to_datetime("now")
            data["update_date"] = pd.to_datetime("now")
            df = pd.DataFrame([data])
            df.to_sql(table_name, self.engine, if_exists='append', index=False)
            print(f"Los datos han sido cargados en la tabla {table_name} de la base de datos.")
        else:
            print(f"La tabla {table_name} no existe en la base de datos.")


    def get_data_filtered(self, table_name: str, filters: dict):
        query = self.session.query(self.models[table_name.capitalize()])
        primary_key = self.models[table_name.capitalize()].__table__.primary_key.columns[0].name
        for key, value in filters.items():
            query = query.filter(getattr(self.models[table_name.capitalize()], key) == value)
        
        # Ejecutar la consulta
        result = query.all()
        
        # Convertir el resultado a un DataFrame
        data = [row.__dict__ for row in result]  # Convierte las filas a diccionarios
        for item in data:
            item.pop('_sa_instance_state', None)  # Elimina metadatos internos de SQLAlchemy

        df = pd.DataFrame(data)
        df = df.set_index(primary_key)
        return df
    
    def modify_data(self, table_name: str, filters: dict, data: dict):
        query = self.session.query(self.models[table_name.capitalize()])
        data["date_update"] = pd.to_datetime("now")
        for key, value in filters.items():
            query = query.filter(getattr(self.models[table_name.capitalize()], key) == value)
        

        # Actualizar los registros con los nuevos valores
        rows_updated = query.update(data, synchronize_session='fetch')
        self.session.commit()

        print(f"{rows_updated} registros han sido actualizados en la tabla {table_name} de la base de datos.")
    
    def get_all_data(self, table_name: str):
        query = self.session.query(self.models[table_name.capitalize()])
        primary_key = self.models[table_name.capitalize()].__table__.primary_key.columns[0].name
        result = query.all()
        
        # Convertir el resultado a un DataFrame
        data = [row.__dict__ for row in result]  # Convierte las filas a diccionarios
        for item in data:
            item.pop('_sa_instance_state', None)  # Elimina metadatos internos de SQLAlchemy

        df = pd.DataFrame(data)
        df = df.set_index(primary_key)
        return df
    def get_data(self, table_name: str):
        query = self.session.query(self.models[table_name.capitalize()])
        primary_key = self.models[table_name.capitalize()].__table__.primary_key.columns[0].name
        result = query.all()
        
        # Convertir el resultado a un DataFrame
        data = [row.__dict__ for row in result]  # Convierte las filas a diccionarios
        for item in data:
            item.pop('_sa_instance_state', None)  # Elimina metadatos internos de SQLAlchemy

        df = pd.DataFrame(data)
        df = df.set_index(primary_key)
        return df
    
    def get_columns(self, table_name: str):
        inspector = inspect(self.engine)
        columns = inspector.get_columns(table_name)
        return columns

In [102]:
bbdd_magement = BBDD_MANAGEMENT('retail_web_jewelry.db')

/var/folders/r1/hd_91nzn1yb8kh1vq1c7gt2w0000gn/T/ipykernel_20206/1830934054.py:4: MovedIn20Warning: The ``declarative_base()`` function is now available as sqlalchemy.orm.declarative_base(). (deprecated since: 2.0) (Background on SQLAlchemy 2.0 at: https://sqlalche.me/e/b8d9)
  self.Base = declarative_base()


In [75]:
bbdd_magement.create_table('jewelry', {'id': Integer, 'name': String, 'price': Integer, 'description': String, 'image': String})

La tabla jewelry ya existe en la base de datos.


In [103]:
bbdd_magement.create_table('imagenes', {"id": String,'name': String, 'size': Integer}, {'name': {'reference_table': 'jewelry', 'reference_column': 'image'}})

La tabla imagenes ha sido creada en la base de datos.


In [105]:
bbdd_magement.upload_data('imagenes', {'name': 'Jewelry 4', 'size': 100})


Los datos han sido cargados en la tabla imagenes de la base de datos.


In [46]:
bbdd_magement.upload_data('jewelry', {'name': 'Jewelry 1', 'price': 100, 'description': 'Description 1', 'image': 'image1.jpg'})
bbdd_magement.upload_data('jewelry', {'name': 'Jewelry 2', 'price': 100, 'description': 'Description 2', 'image': 'image2.jpg'})

Los datos han sido cargados en la tabla jewelry de la base de datos.
Los datos han sido cargados en la tabla jewelry de la base de datos.


In [67]:
bbdd_magement.get_data_filtered('jewelry', {'name': 'Jewelry 1'})

,description,name,create_date,price,image,update_date
id,,,,,,
1,Description 1,Jewelry 1,2024-11-17 22:01:29.373864,100,image1.jpg,2024-11-17 22:01:29.374207


In [68]:
bbdd_magement.get_all_data('jewelry')

,description,name,create_date,price,image,update_date
id,,,,,,
1,Description 1,Jewelry 1,2024-11-17 22:01:29.373864,100,image1.jpg,2024-11-17 22:01:29.374207
2,Description 2,Jewelry 2,2024-11-17 22:01:29.388742,100,image2.jpg,2024-11-17 22:01:29.389160


In [12]:
a = pd.DataFrame([{'name': 'Jewelry 3', 'price': 100, 'description': 'Description 3', 'image': 'image3.jpg'}])
new_row = {'name': 'Jewelry 4', 'price': 150, 'description': 'Description 4', 'image': 'image4.jpg'}

a = pd.concat([a, pd.DataFrame([new_row])], ignore_index=True)
a.set_index("name", inplace=True)
a.loc["Jewelry 4"]
# a

price                    150
description    Description 4
image             image4.jpg
Name: Jewelry 4, dtype: object

In [13]:
a.loc["Jewelry 4"].to_dict()

{'price': 150, 'description': 'Description 4', 'image': 'image4.jpg'}

In [2]:
from utils.languages_bbdd_manager import LANGUAJE_BBDD_MANAGEMENT



languajes_manager = LANGUAJE_BBDD_MANAGEMENT("lenguage_bbdd")

In [5]:
languajes_manager.create_table_languajes()

La tabla languajes ha sido creada en la base de datos.


In [7]:
languajes_manager.add_data(data={'keywords': 'caca', 'word_es': 'pedo', 'word_en': 'pis'})

Los datos han sido cargados en la tabla languajes de la base de datos.


In [3]:
languajes_manager.get_language_data()

'{"word_en":{"jewelry":"jewelry","caca":"pis"},"word_es":{"jewelry":"joyeria","caca":"pedo"}}'

In [9]:
a[["word_en", "word_es"]].to_json()

'{"word_en":{"jewelry":"jewelry","caca":"pis"},"word_es":{"jewelry":"joyeria","caca":"pedo"}}'